In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable

In [0]:
dbutils.widgets.text('incremental_flag', '0')

In [0]:
incremental_flag = dbutils.widgets.get('incremental_flag')
print(incremental_flag)

In [0]:
df_src = spark.sql("""
select DISTINCT (Model_ID) AS Model_id, Model_Category from parquet.`abfss://silver@swap01storageaccount.dfs.core.windows.net/carsalesdata`
""")


df_src.display()

In [0]:
if spark.catalog.tableExists('bmw_cars_catalog.gold.model'):
    df_sink = spark.sql('''
    select dim_model_key, Model_id, Model_Category from parquet.`abfss://silver@swap01storageaccount.dfs.core.windows.net/carsalesdata`
    where 1 = 0 ''')
else:
    df_sink = spark.sql('''
    select 1 as dim_model_key, Model_id, Model_Category from parquet.`abfss://silver@swap01storageaccount.dfs.core.windows.net/carsalesdata`
    where 1 = 0 ''')



df_sink.display()

In [0]:
df_filter = df_src.join(df_sink, df_src['Model_id'] == df_sink['Model_id'], 'left').select(df_src['Model_id'], df_src['Model_Category'], df_sink['dim_model_key'])
                                                                                                                                                 
df_filter.display()

In [0]:
df_filter_old = df_filter.filter(col('dim_model_key').isNotNull())

display(df_filter_old)

In [0]:
df_filter_new = df_filter.filter(col('dim_model_key').isNull()).select(df_src['Model_id'], df_src['Model_Category'])

display(df_filter_new)

In [0]:
if (incremental_flag == '0'):
    max_value = 1
else:
    max_value = spark.sql('select max(dim_model_key) from bmw_cars_catalog.gold.model').collect()[0][0]


In [0]:
df_filter_new = df_filter_new.withColumn('dim_model_key', max_value + monotonically_increasing_id())
df_filter_new.display()


In [0]:
df_final = df_filter_new.union(df_filter_old)
df_final.display()

In [0]:
#Incremental load
if spark.catalog.tableExists('bmw_cars_catalog.gold.model'):
    delta_tbl = DeltaTable.forPath(spark, 'abfss://silver@swap01storageaccount.dfs.core.windows.net/dim_model')
    delta_tbl.alias('t').merge(df_final.alias('s'), 't.Model_id = s.Model_id')\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
# Initial Run
else:
    df_final.write.format("delta")\
        .mode('overwrite')\
        .option('path', 'abfss://silver@swap01storageaccount.dfs.core.windows.net/dim_model')\
        .saveAsTable('bmw_cars_catalog.gold.model')